In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.0f}'.format

In [2]:
nurse_df = pd.read_csv('https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/cdph-skilled-nursing-totals.csv')
nurse_df.head(5)

,date,staff_active_cases,patients_active_cases,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,staff_deaths_note
0,2021-12-28,373,1194,56927,66144,252,9387,NaN
1,2021-12-27,271,940,56657,66049,252,9382,NaN
2,2021-12-26,304,816,56522,66009,252,9382,NaN
3,2021-12-25,236,734,56428,65962,252,9381,NaN
4,2021-12-24,245,638,56309,65922,252,9380,NaN


In [3]:
nurse_df.drop(nurse_df.columns[[1, 2, 7]], axis=1, inplace=True)
nurse_df['type'] = 'SN'
nurse_df

,date,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,type
0,2021-12-28,56927,66144,252,9387,SN
1,2021-12-27,56657,66049,252,9382,SN
2,2021-12-26,56522,66009,252,9382,SN
3,2021-12-25,56428,65962,252,9381,SN
4,2021-12-24,56309,65922,252,9380,SN
...,...,...,...,...,...,...
569,2020-04-29,2594,3923,19,618,SN
570,2020-04-28,2323,3515,1,578,SN
571,2020-04-25,2329,3441,1,545,SN
572,2020-04-24,2099,3141,1,486,SN


In [4]:
today = nurse_df[(nurse_df['date'] == '2021-12-28')]
today.set_index('date', inplace=True)
today

,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,type
date,,,,,
2021-12-28,56927,66144,252,9387,SN


In [5]:
senior_df = pd.read_csv('https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/cdph-adult-and-senior-care-totals.csv')

In [6]:
senior_df

,date,confirmed_cases,deaths,active_cases,source_url
0,2021-07-27,43066,3749,307,https://www.cdss.ca.gov/Portals/9/Additional-R...
1,2021-07-14,42910,3747,181,https://www.cdss.ca.gov/Portals/9/Additional-R...
2,2021-07-12,42811,3746,97,https://www.cdss.ca.gov/Portals/9/Additional-R...
3,2021-06-21,42733,3744,59,https://www.cdss.ca.gov/Portals/9/Additional-R...
4,2021-05-31,42696,3744,89,https://www.cdss.ca.gov/Portals/9/Additional-R...
...,...,...,...,...,...
290,2020-05-02,1404,175,nan,https://www.cdss.ca.gov/Portals/9/Additional-R...
291,2020-05-01,1340,170,nan,https://www.cdss.ca.gov/Portals/9/Additional-R...
292,2020-04-29,1267,155,nan,https://www.cdss.ca.gov/Portals/9/Additional-R...
293,2020-04-28,1198,147,nan,https://www.cdss.ca.gov/Portals/9/Additional-R...


In [7]:
senior_df.drop(senior_df.columns[[3, 4]], axis=1, inplace=True)
senior_df['type'] = 'SC'
sc_today = senior_df[(senior_df['date'] == '2021-07-27')]
sc_today

,date,confirmed_cases,deaths,type
0,2021-07-27,43066,3749,SC


In [8]:
sc_today.set_index('date', inplace=True)
sc_today

,confirmed_cases,deaths,type
date,,,
2021-07-27,43066,3749,SC


In [9]:
latest_df = pd.merge(today, sc_today, left_on=['date'], right_on=['date'])
latest_df

,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,type_x,confirmed_cases,deaths,type_y
date,,,,,,,,


In [10]:
combined = today.merge(sc_today, how='outer', left_index=True, right_index=True)
combined

,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,type_x,confirmed_cases,deaths,type_y
date,,,,,,,,
2021-07-27,nan,nan,nan,nan,NaN,43066,3749,SC
2021-12-28,56927,66144,252,9387,SN,nan,nan,NaN


In [11]:
combined['patient_cases'] = combined["patients_confirmed_cases"] + combined["confirmed_cases"]
combined['patient_deaths'] = combined["patients_deaths"] + combined["deaths"]
combined

,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,type_x,confirmed_cases,deaths,type_y,patient_cases,patient_deaths
date,,,,,,,,,,
2021-07-27,nan,nan,nan,nan,NaN,43066,3749,SC,nan,nan
2021-12-28,56927,66144,252,9387,SN,nan,nan,NaN,nan,nan


In [12]:
combined2 = combined[['patient_cases', 'patient_deaths', 'staff_confirmed_cases', 'staff_deaths']]
combined2

,patient_cases,patient_deaths,staff_confirmed_cases,staff_deaths
date,,,,
2021-07-27,nan,nan,nan,nan
2021-12-28,nan,nan,56927,252
